In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum, LinExpr
import pandas as pd
import numpy as np
from utils.is_efficient import is_efficient
from models.modelsDEA import DEA
from utils.groupDMUs import groupDMUs


In [2]:
#RDM
N1=list(range(0,101))
N2=list(range(100,201))
R1=[0]
R2=[1,2]
N1_bar=list(range(0,201))
N2_bar=list(range(0,101))
m=3
q=3
n=200


alphas=[]
betas=[]
Results=[]
lambdas_1=[]

for o in N1:
    alpha = {}
    beta = {}
    lambdas = {}
    model = Model("DMU_Optimization")
    for i in range(m):
        for j in N1:
            alpha[i, j] = model.addVar(vtype=GRB.CONTINUOUS, lb=0.6, ub=0.85, name=f"alpha_{i}_{j}_0")
    beta[o, 0] = model.addVar(vtype=GRB.CONTINUOUS, lb=0,ub=1, name=f"beta_{j}_0")
    beta[o, 1] = model.addVar(vtype=GRB.CONTINUOUS,lb=0,ub=1, name=f"beta_{j}_1")

    for j in N1_bar:
        lambdas[j, 0] = model.addVar(vtype=GRB.CONTINUOUS,lb=0, ub=1, name=f"lambda_{j}_0")
    for j in N2_bar:
        lambdas[j, 1] = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f"lambda_{j}_1")

 
    model.setParam('DualReductions', 0)
    model.setObjective(
   (beta[o, 0] + beta[o,1])/2, GRB.MAXIMIZE
)
   
    
    R_minus = [x[o, i] - min(x[:, i]) for i in range(m)]
    R_plus = [max(y[:, r]) - y[o, r] for r in range(q)]
     

    model.addConstr(
            quicksum(lambdas[j, 0] * y[j,0] for j in N1_bar) >= y[o,0] + beta[o, 0] * R_plus[0]
        )
    model.addConstr(
            quicksum(lambdas[j, 1] * y[j,1] for j in N2_bar) >= y[o,1] + beta[o, 1] * R_plus[1]
        )
    model.addConstr(
            quicksum(lambdas[j, 1] * y[j,2] for j in N2_bar) >= y[o,2] + beta[o, 1] * R_plus[2]
        )
    

    for i in range(m):  

        model.addConstr(
                quicksum(lambdas[j, 0] * alpha[i, j] * x[j,i] for j in N1) + quicksum(lambdas[j, 0] * x[j,i] for j in N2)
                <= alpha[i, o] * x[o,i] - beta[o, 0] * alpha[i, o] * R_minus[i]
            )
        
        model.addConstr(
                quicksum(lambdas[j, 1] * (1-alpha[i, j]) * x[j,i] for j in N1)
                <= (1-alpha[i, o]) * x[o,i] - beta[o, 1] * (1-alpha[i, o]) * R_minus[i]
            )
        
    model.addConstr(quicksum(lambdas[j, 0] for j in N1_bar) == 1)
    model.addConstr(quicksum(lambdas[j, 1] for j in N2_bar) == 1)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        alpha_values1 = model.getAttr("x", alpha)
        beta_values1 = model.getAttr("x", beta)
        lambda_values1 = model.getAttr("x", lambdas)
        for key, value in alpha_values1.items():
            if key[1]==o:
                    alphas.append({
                            "DMU": o,
                            "alphas_RDM": {'key':key, 'value': value}
                        })
        for key,value in lambda_values1.items():
            lambdas_1.append({
                            "DMU": o,
                            "lambda": {'key':key, 'value': value}
                        })
        for key, value in beta_values1.items():
            if key[0]==o:
                betas.append({
                            "DMU": o,
                            "betas": {'key':key, 'value': value}
                        })
                
        objective_value = model.objVal
        Results.append({"DMU":o, "efficiency_RDM":1-objective_value})
    else:
        print("No optimal solution found.")


dmu_values = {}
for item in alphas:
    dmu = item['DMU']
    value = item['alphas_RDM']['value']
    dmu_values.setdefault(dmu, []).append(value)
values_list = list(dmu_values.values())



for o in N2:
    alpha = {}
    beta = {}
    lambdas = {}
    model = Model("DMU_Optimization")
    for i in range(m):
        for j in N1:
            alpha[i, j] = model.addVar(vtype=GRB.CONTINUOUS, lb=0.6, ub=0.85, name=f"alpha_{i}_{j}_0")
    beta[o, 0] = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1,name=f"beta_{j}_0")

    for j in N1_bar:
        lambdas[j, 0] = model.addVar(vtype=GRB.CONTINUOUS,lb=0, ub=1,name=f"lambda_{j}_0")

    model.setParam('DualReductions', 0)
    model.setObjective(
   (beta[o, 0]), GRB.MAXIMIZE
)
    
    R_minus = [x[o, i] - min(x[:, i]) for i in range(m)]
    R_plus = [max(y[:, r]) - y[o, r] for r in range(q)]

    model.addConstr(
            quicksum(lambdas[j, 0] * y[j,0] for j in N1_bar) >= y[o,0] + beta[o, 0] * R_plus[0]
        )
    
    for i in range(m):  

        model.addConstr(
                quicksum(lambdas[j, 0] * values_list[j][i] * x[j,i] for j in N1) + quicksum(lambdas[j, 0] * x[j,i] for j in N2)
                <=  x[o,i] - beta[o, 0]* R_minus[i]
            )
        
    model.addConstr(quicksum(lambdas[j, 0] for j in N1_bar) == 1)


   
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        alpha_values1 = model.getAttr("x", alpha)
        beta_values1 = model.getAttr("x", beta)
        lambda_values1 = model.getAttr("x", lambdas)
        for key, value in alpha_values1.items():
            if key[1]==o:
                    alphas.append({
                            "DMU": o,
                            "alphas": {'key':key, 'value': value}
                        })
        for key,value in lambda_values1.items():
            lambdas_1.append({
                            "DMU": o,
                            "lambda": {'key':key, 'value': value}
                        })
        for key, value in beta_values1.items():
            if key[0]==o:
                betas.append({
                            "DMU": o,
                            "betas": {'key':key, 'value': value}
                        })
                
        objective_value = model.objVal
        Results.append({"DMU":o, "efficiency_RDM":1-objective_value})
    else:
        print("No optimal solution found.")
Results

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2579375
Academic license 2579375 - for non-commercial use only - registered to ma___@iitd.ac.in
Set parameter DualReductions to value 0


NameError: name 'x' is not defined

In [1]:
import gurobipy
gurobipy.gurobi._getdefaultenv()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2579375
Academic license 2579375 - for non-commercial use only - registered to ma___@iitd.ac.in


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=2579375>